# Extracting Outlines from PDFs

The objective of this piece of code is to go through all the PDFs and detect if there are outlines present. The output of this code is the following: <br>

1. Outline_Present.csv - This has the list of the PDF files and then there is an indicator is the outline i spresent or not. <br>

2. Index 4 - Chapters.csv - This table has a detailed list of all the chapters present in all the PDF files present. 

In [184]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [185]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1328

## Extract Outlines for all the PDFs

In [190]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, file_names_ch, chapters, levels, pages)
    else:
        chapters.extend([s.title])
        levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        try:
            pages.extend([pdfread.getDestinationPageNumber(s)])
        except: 
            pages.extend([-999])
        file_names_ch.append([file_name])

In [197]:
# Varibles for file outline_present.csv
file_names = []
outline_present = []

# Variables for Index 4 - Chapters.csv
file_names_ch = []
chapters = []
levels = []
pages = []


#pdf_paths = pdf_paths[0:10]

for pdf_path in pdf_paths:
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    try:
        pdfread = p2.PdfFileReader(pdf_path)
        s = pdfread.outlines
        len_s = len(s)
    except:
        outline_present.append("Error in reading the outline")
        
    if len_s > 0:
        level = 0
        outline_present.append(1)
        nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages)
    elif len_s == 0:
        outline_present.append(0)
    
    # Error Code for s (=pdfraed.outlines) is -999
    # the negative number allows exclusivity from the if and the elif conditions above
    len_s = -999 

#### Saving the file outline_present.csv

In [203]:
len(file_names)
len(pdf_paths)
len(outline_present)

1328

1328

1328

In [204]:
df_outlines = pd.DataFrame({'file_names': file_names, 'pdf_paths' : pdf_paths, 'outline_present' : outline_present})
df_outlines#.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_names,pdf_paths,outline_present
0,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
2,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0
3,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0
4,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,0
...,...,...,...
1323,A6T2W2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
1324,A6T2W3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
1325,A6T2W4.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1
1326,A6T2W5.pdf,F:/Environmental Baseline Data/Version 3/Data/...,1


#### Saving the file Index 4 - Chapters.csv

In [205]:
len(file_names_ch)
len(chapters)
len(pages)
len(levels)

19439

19439

19439

19439

In [206]:
df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})
df_chapters.head(5)
df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index 4 - Chapters.csv")

,file_names,chapters,pages,levels
0,[A0H8C0.pdf],Environmental Impact Assessment and Mitigation...,24,2
1,[A0H8C0.pdf],Introduction,24,3
2,[A0H8C0.pdf],Project Background,24,4
3,[A0H8C0.pdf],Regulatory Setting,24,4
4,[A0H8C0.pdf],,-1,5


## Is Table of contents Present?